# Hyperparameter Optimization
We load the 5 random train test splits and for each train test split we optimize the hyper parameters and choose the ones with the highest recall. Then we calculate the mean recall an ndcg and the standard deviations between the 5 train test splits and look at the optimal hyperparameters for each spli.

In [7]:
# set to the correct paths
import warnings
warnings.filterwarnings("ignore")
allData=False


path="/home/kristof/Desktop/AI_Project/"


# path="./drive/MyDrive/AI Project/"


interactions_csv_path = path+"datasets/TestDataPreprocessed_interactions.csv"
# goodreads_path = '/home/andy/Desktop/AI_Project/datasets/Goodreads/'
folds_path=path+"datasets/"
consgroup_path=path+"datasets/TestDataConsGroupdict.pickle"
reversegroup_path=path+"datasets/TestDataReverseGroupdict.pickle"
groupdict_path=path+"datasets/TestDataGroupdict.pickle"
userdic_path=path+"datasets/TestDataUser_dic.pickle"
books_df_g_path=path+"datasets/books_df_grouped.csv"


In [8]:
# Mount google drive
# from google.colab import drive
# drive.mount('/content/drive')

# EASER modified to work with groups
Easer normally sets the self similarity to zero to avoid items recommending themselves, but when all volumes are combined into one group we do want those groups to be similar to themselves so we set the diagonal to zeros except for the possitions of the grouped volumes


Calculating the inverse of the dense item to item matrix is very expensive memory wise. To combat this we use a sparse approximation https://arxiv.org/pdf/1904.13033.pdf#page=6

In [9]:

#Embarrassingly Shallow Autoencoder
from scipy.sparse import find
import itertools
import re
import numpy as np
class Ease():
    def __init__(self, K=10, l=1000):
        #how many predictions to make
        self.K = K
        #lambda L2 regularization term

        self.l = l
        #weight matrix B
        self.B = None
        #threshold for corrcoeff matrix in fit
        self.threshold = 0.65
        #new diag that helps for adjusted self-similarity scores
        self.newDiag = []

    def fit(self, X):
        #implementation of sparse inverse calculations
        #get ItemxItem gram matirx G
        G = (X.T @ X).toarray().astype(np.float32)
        #calculate corrcoef matrix on G
        c=np.corrcoef(G).astype(np.float32)
        c = np.nan_to_num(c)
        
        #set threshold for maximal size of blocks
        maxi=1000
        # threshold=0.3 -> 850 items worden gerecommend
        # maxi=1500
        #first set all items in the corrcoef matrix that are above a threshold to 1
        for row in c.T:
          l=list(reversed(np.argsort(row)))[:maxi]
          for i in l:
            if row[i]>self.threshold:
              row[i]=1
            else:
              break
        #binarize corrcoef matrix
        A=np.where(c==1, 1, 0)
        

        #get list of indices in descending order of column sums
        res=A.sum(axis=0)
        
        l=np.array(list(reversed(np.argsort(res))))
        setje=set()
        #iterate over indices and generate submatrices
        spars=scipy.sparse.lil_matrix(G.shape)
        for i in l:
          if i in setje:
              continue
          #get itemindexes where element == 0
          col=A[:,i]
          itemindexes=np.where(col==1)
          #construct submatrix
          sub=G[np.ix_(itemindexes[0],itemindexes[0])]
          diag=scipy.sparse.dia_matrix(sub.shape)
          #set submatrixs diagonal to lambda + submatrix
          diag.setdiag([self.l]*sub.shape[0])
          sub2=(sub+diag)
          #calculate inverse and set diagonal to 0 (self-similarity)
          P=np.linalg.inv(sub2)
          B=-P/P.diagonal()
         
          
          np.fill_diagonal(B,0)
          # store values in sparse matrix
          for i in itertools.product(range(B.shape[0]),range(B.shape[0])):
            i1=i[0]
            j1=i[1]
            spars[itemindexes[0][i1],itemindexes[0][j1]]=B[i1,j1]
          for j in itemindexes[0]:
              setje.add(j)
        # d = [0 if len(reverseGroupdic[key])==1 else similarityScoreDic[key] for key in similarityScoreDic]
        spars.setdiag(self.newDiag)
        self.B = spars.toarray()
    def predict(self, X, firstUserID):
        
        S = X*self.B
        users = set(X.nonzero()[0])
        
      
        U, I, V = [], [], []
        real_groups=list(cons_group_dic.keys())

        for user in users:
            history_of_user=randomSplit_df.loc[randomSplit_df['user_id']==user+firstUserID].history.item()
            U.extend([user] * self.K)
            

            # a=np.array([S[user][i] if i not in history_of_user or (len(reverseGroupdic[i]) > 1 and len(re.split(' Volume| Vol\.| vol\.| #', books_df_g.loc[books_df_g['group_id'] == list(cons_group_dic.keys())[i]].title.item())) > 1) else -self.l*2 for i in range(len(S[user])) ])
            a = []
            for i in range(len(S[user])):
             
              try:
                lastViewedIndex = (len(history_of_user) - 1 - history_of_user[::-1].index(i))
              except:
                lastViewedIndex = len(history_of_user)
              if i not in history_of_user or (len(reverseGroupdic[real_groups[i]]) > 1 and len(re.split(' Volume| Vol\.| vol\.| #', books_df_g.loc[books_df_g['group_id'] == real_groups[i]].title.item())) > 1):
                a.append(S[user][i]*((lastViewedIndex/len(history_of_user))**2)) 
              else:
                a.append(-self.l*2)
            # a = np.array([S[user][i]*history_of_user.count(i) if (len(reverseGroupdic[i]) > 1 and history_of_user.count(i)<len(reverseGroupdic[i])) else a[i] for i in range(len(a))])
            items = np.array(a).argsort()[-self.K:][::-1]
            I.extend(items)
            V.extend(S[user][items])

        score_matrix = scipy.sparse.csr_matrix((V, (U, I)), shape=X.shape).astype(np.float32)
        return score_matrix



# Hyperparameter Optimization For each split

In [10]:
from scipy.sparse import vstack
from sklearn.model_selection import KFold
import sklearn.metrics
import statistics
import pickle
import pandas as pd
with open(consgroup_path, 'rb') as f:
    cons_group_dic= pickle.load(f)
with open(groupdict_path, 'rb') as f:
    groupdic= pickle.load(f)
with open(reversegroup_path, 'rb') as f:
    reverseGroupdic= pickle.load(f)
with open(userdic_path, 'rb') as f:
    userdic= pickle.load(f)
randomSplit_df=0
books_df_g=pd.read_csv(books_df_g_path)
books_df_g["group_id"]=range(0,len(books_df_g))

def recall_easer_hyperparameter_optimization(model,K):
  global randomSplit_df

  ease = model
  recalls = []
  ndcgs = []
  y_pred_totals = []
  hyperParams=[]
  for s in range(1,6):
    if allData:
      randomSplit_df = pd.read_csv(folds_path+"AllDatafold"+str(s)+".csv")
    else:
      randomSplit_df = pd.read_csv(folds_path+"TestDatafold"+str(s)+".csv")
    randomSplit_df['user_id'] = randomSplit_df.user_id.apply(lambda x:int(userdic[int(x)]))

    randomSplit_df['history'] = randomSplit_df.history.apply(lambda x:[int(cons_group_dic[groupdic[int(y)]]) if y else y for y in x[1:-1].split(',')])
    randomSplit_df['future'] = randomSplit_df.future.apply(lambda x:[int(cons_group_dic[groupdic[int(y)]]) if y else y for y in x[1:-1].split(',')])
    shape = (randomSplit_df['user_id'].max() +1, len(cons_group_dic))

    ease.newDiag = get_newDiag(randomSplit_df,shape)
    train_x = create_sparse_matrix(randomSplit_df, column='history', shape=shape)
    y_true = create_sparse_matrix(randomSplit_df, column='future', shape=shape)
   
    
    recalls_fold = []
    ndcgs_fold=[]
    hyperParams_fold=[]
    for lambda_ in [250,5000,50000]:
      for threshold in [0.8,0.9]:
        ease.threshold = threshold
        ease.l=lambda_
        ease.fit(train_x)

        interval=int(train_x.shape[0]/20)
        true_ndcg=0
        true_recall=0
        y_pred_total = scipy.sparse.csr_matrix((0,train_x.shape[1])).astype(np.float32)
        
        for j in range(20-1):
          predict_split=train_x[np.ix_(list(range(interval*j,interval*(j+1))),list(range(train_x.shape[1])))]
          true_split=y_true[np.ix_(list(range(interval*j,interval*(j+1))),list(range(train_x.shape[1])))]
          y_pred = ease.predict(predict_split, interval*j)
          y_pred_total = vstack((y_pred_total, y_pred))
          scores = scipy.sparse.lil_matrix(y_pred.shape)
          scores[y_pred.multiply(true_split).astype(bool)] = 1
          scores = scores.tocsr()
          scores = sparse_divide_nonzero(scores, scipy.sparse.csr_matrix(true_split.sum(axis=1))).sum(axis=1)
          true_recall+=scores.mean()*(interval/train_x.shape[0])
      
          ndcg = sklearn.metrics.ndcg_score(true_split.toarray(), y_pred.toarray(), k=K)
          true_ndcg+=ndcg*(interval/train_x.shape[0])

        predict_split=train_x[np.ix_(list(range(interval*19,train_x.shape[0])),list(range(train_x.shape[1])))]
        true_split=y_true[np.ix_(list(range(interval*19,train_x.shape[0])),list(range(train_x.shape[1])))]
        y_pred = ease.predict(predict_split, interval*19)
        y_pred_total = vstack((y_pred_total, y_pred))
        scores = scipy.sparse.lil_matrix(y_pred.shape)
        scores[y_pred.multiply(true_split).astype(bool)] = 1
        scores = scores.tocsr()
        scores = sparse_divide_nonzero(scores, scipy.sparse.csr_matrix(true_split.sum(axis=1))).sum(axis=1)
        true_recall+=scores.mean()*(interval/train_x.shape[0])

        
        ndcg = sklearn.metrics.ndcg_score(true_split.toarray(), y_pred.toarray(), k=K)
        true_ndcg+=ndcg*(interval/train_x.shape[0])
        print("recall @ {}: {:.4f}, ndcg: {:.4f} params lambda:{}, threshold={}, seed= {}".format(K,true_recall,true_ndcg ,lambda_, threshold,s))

        
        ndcgs_fold.append(true_ndcg)
        recalls_fold.append(true_recall)
        hyperParams_fold.append((lambda_, threshold))
    max_value = max(recalls_fold)
    max_index = recalls_fold.index(max_value)
    recalls.append(recalls_fold[max_index])
    ndcgs.append(ndcgs_fold[max_index])
    hyperParams.append(hyperParams_fold[max_index])
  
  meanRecall = statistics.mean(recalls)
  stdevRecall = statistics.stdev(recalls)

  meanNdcg = statistics.mean(ndcgs)
  stdevNdcg = statistics.stdev(ndcgs)
  print("mean recall @ {}: {:.4f}, stdev: {:.4f}".format(K, meanRecall, stdevRecall))
  print("mean ndcg @ {}: {:.4f}, stdev: {:.4f}".format(K, meanNdcg, stdevNdcg))
  print("used hyperParameters (lambda, threshold): {}".format(hyperParams))
  return meanNdcg,meanRecall, hyperParams
  

In [11]:
import scipy.sparse

#Create scipy csr matrix
def create_sparse_matrix(sessions_df, column='history', shape=None):
  #flatten
  user_ids = []
  item_ids = []
  for idx, row in sessions_df.iterrows():
    items = row[column]
    user = row['user_id']
    user_ids.extend([user] * len(items))
    item_ids.extend(items)
  #create csr matrix
  values = np.ones(len(user_ids))

  matrix = scipy.sparse.csr_matrix((values, (user_ids, item_ids)), shape=shape, dtype=np.int32)
  return matrix


def get_newDiag(df,shape):
  self_similarity_n={}
  self_similarity={}
  for idx,row in df.iterrows():
    l=row['history']
    histset=set(l)
    real_group_id_list=list(cons_group_dic.keys())
    for concgroup in histset:
      # print(concgroup,histset)
      if not concgroup:
        continue
      real_group_id=real_group_id_list[concgroup]
      grouplist=reverseGroupdic[real_group_id]
      if len(grouplist) ==1:
        self_similarity[concgroup]=0
        # self_similarity_n[concgroup]=1

      else:
        self_similarity[concgroup]=1
  # for i in self_similarity:
  #   self_similarity[i]=self_similarity[i]/(self_similarity_n[i]+20)
  new_diag=[self_similarity[i] if i in self_similarity else 0 for i in list(range(shape[1]))]
  return new_diag

In [12]:
def sparse_divide_nonzero(a: scipy.sparse.csr_matrix, b: scipy.sparse.csr_matrix) -> scipy.sparse.csr_matrix:
    return a.multiply(sparse_inverse_nonzero(b))

def sparse_inverse_nonzero(a: scipy.sparse.csr_matrix) -> scipy.sparse.csr_matrix:
    inv_a = a.copy()
    inv_a.data = 1 / inv_a.data
    return inv_a


K = 10
for K in [10]:
  
  ease = Ease(K=K)
  ndcg,recall, y_pred =recall_easer_hyperparameter_optimization(ease,K)

recall @ 10: 0.1899, ndcg: 0.4558 params lambda:250, threshold=0.8, seed= 1
recall @ 10: 0.2012, ndcg: 0.4665 params lambda:250, threshold=0.9, seed= 1
recall @ 10: 0.1966, ndcg: 0.4673 params lambda:5000, threshold=0.8, seed= 1
recall @ 10: 0.2192, ndcg: 0.4887 params lambda:5000, threshold=0.9, seed= 1
recall @ 10: 0.2238, ndcg: 0.4965 params lambda:50000, threshold=0.8, seed= 1
recall @ 10: 0.2449, ndcg: 0.5129 params lambda:50000, threshold=0.9, seed= 1
recall @ 10: 0.1830, ndcg: 0.4449 params lambda:250, threshold=0.8, seed= 2
recall @ 10: 0.1920, ndcg: 0.4519 params lambda:250, threshold=0.9, seed= 2
recall @ 10: 0.1859, ndcg: 0.4515 params lambda:5000, threshold=0.8, seed= 2
recall @ 10: 0.2035, ndcg: 0.4686 params lambda:5000, threshold=0.9, seed= 2
recall @ 10: 0.2060, ndcg: 0.4754 params lambda:50000, threshold=0.8, seed= 2
recall @ 10: 0.2264, ndcg: 0.4903 params lambda:50000, threshold=0.9, seed= 2
recall @ 10: 0.2169, ndcg: 0.4784 params lambda:250, threshold=0.8, seed= 3
